In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import folium
import geojson
import numpy as np
from folium import plugins

KeyboardInterrupt: 

In [ ]:
from folium.elements import JSCSSMixin
from folium.map import Layer
from jinja2 import Template

class FeatureGroupSubGroup(JSCSSMixin, Layer):
    _template = Template(u"""
        {% macro script(this, kwargs) %}
            var {{ this.get_name() }} = L.featureGroup.subGroup(
                {{ this._group.get_name() }}
            );
            {{ this.get_name() }}.addTo({{ this._parent.get_name() }});
        {% endmacro %}
        """)

    default_js = [
        ('featuregroupsubgroupjs',
         'https://unpkg.com/leaflet.featuregroup.subgroup@1.0.2/dist/leaflet.featuregroup.subgroup.js'),
    ]

    def __init__(self, group, name=None, overlay=True, control=True, show=True):
        super(FeatureGroupSubGroup, self).__init__(name=name, overlay=overlay,
                                                   control=control, show=show)

        self._group = group
        self._name = 'FeatureGroupSubGroup'

In [ ]:
"""
ray_17_04 = pd.read_excel('Rayos_2017-04.xlsx')
ray_17_04 = ray_17_04[["LATITUD","LONGITUD","TIPO", "CORRIENTE", "ERROR"]]
ray_17_04 = ray_17_04.rename(columns={"LONGITUD":"COOR_X", "LATITUD":"COOR_Y"})

ray_18_jul = pd.read_excel('Rayos_julio_2018_CODENSA.xlsx')
ray_18_jul = ray_18_jul[["Latitud","Longitud","Tipo", "Corriente_[kA]", "Error_[km]"]]
ray_18_jul = ray_18_jul.rename(columns={"Longitud":"COOR_X", "Latitud":"COOR_Y", "Corriente_[kA]":"CORRIENTE", "Error_[km]":"ERROR"})

ray_18_jun = pd.read_excel('Rayos_junio_2018_CODENSA.xlsx')
ray_18_jun = ray_18_jun[["Latitud","Longitud","Tipo", "Corriente_[kA]", "Error_[km]"]]
ray_18_jun = ray_18_jun.rename(columns={"Longitud":"COOR_X", "Latitud":"COOR_Y", "Corriente_[kA]":"CORRIENTE", "Error_[km]":"ERROR"})

ray_18_may = pd.read_excel('Rayos_mayo_2018_CODENSA.xlsx')
ray_18_may = ray_18_may[["Latitud","Longitud","Tipo", "Corriente_[kA]", "Error_[km]"]]
ray_18_may = ray_18_may.rename(columns={"Longitud":"COOR_X", "Latitud":"COOR_Y", "Corriente_[kA]":"CORRIENTE", "Error_[km]":"ERROR"})

"""

In [ ]:
#Procesar las Hojas de datos para extraer la información de ubicación necesaria

#----CAMBIAR----#
ray_18_ago = pd.read_excel('Rayos_agosto_2018_CODENSA.xlsx')
ray_18_ago = ray_18_ago[["LATITUD","LONGITUD","TIPO", "CORRIENTE", "ERROR"]]
ray_18_ago = ray_18_ago.rename(columns={"LONGITUD":"COOR_X", "LATITUD":"COOR_Y"})
#----!!CAMBIAR!!----#
gdf = gpd.GeoDataFrame(ray_18_ago, geometry = gpd.points_from_xy(ray_18_ago.COOR_X, ray_18_ago.COOR_Y))
#----!!CAMBIAR!!----#
with open('ray_18_ago.geojson', 'w', encoding='utf-8') as f:
    geojson.dump(gdf, f, ensure_ascii=False)

In [ ]:
bog = gpd.read_file('upz_bogota.geojson')
cun = gpd.read_file('cundi_municipios.geojson')

map = folium.Map(location=[45, -74], zoom_start=8, tiles="Stamen Terrain")

df = gpd.read_file('base_maestra.geojson')
#----!!CAMBIAR!!----#
df2 = gpd.read_file('ray_18_ago.geojson')

#codigo para permitir que sea leido por geojson
df = df.set_crs("EPSG:4326")
df = df.set_crs(epsg=4326)
df2 = df2.set_crs("EPSG:4326")
df2 = df2.set_crs(epsg=4326)


In [ ]:
#----CAMBIAR----#
df2 = gpd.read_file('ray_18_ago.geojson')

ray_indi_mala = df2[abs(df2['CORRIENTE']) > 50]
ray_indi_buena = df2[abs(df2['CORRIENTE']) < 50]

#Agregar columna "color" a los datos de rayos
df2['color'] = "black"

pd.options.mode.chained_assignment = None
for i in range(0,len(df2)):
    if abs(df2.iloc[i]['CORRIENTE']) > 50:
         df2.at[i,'color'] = "red"
    elif abs(df2.iloc[i]['CORRIENTE']) <= 50:
         df2.at[i,'color'] = "green"

In [ ]:
#----CAMBIAR----#
with open('ray_18_ago.geojson', 'w', encoding='utf-8') as f:
    geojson.dump(df2, f, ensure_ascii=False)

In [ ]:
"""
grupo_rc = folium.FeatureGroup(name="RC")
grupo_sz = folium.FeatureGroup(name="SZ")
"""
grupo_bog = folium.FeatureGroup(name="Bogotá dividido por UPZ")
grupo_cun = folium.FeatureGroup(name="Municipios de Cundinamarca")
grupo_ray = folium.FeatureGroup(name="Rayos")
"""
grupo = folium.plugins.FeatureGroupSubGroup( grupo_rc, "Disponibilidad buena RC")
grupo2 = folium.plugins.FeatureGroupSubGroup( grupo_rc, "Disponibilidad mala RC")
grupo3 = folium.plugins.FeatureGroupSubGroup( grupo_sz, "Disponibilidad buena SZ")
grupo4 = folium.plugins.FeatureGroupSubGroup( grupo_sz, "Disponibilidad mala SZ")
"""
grupo5 = folium.plugins.FeatureGroupSubGroup( grupo_ray, "Mucho Riesgo")
grupo6 = folium.plugins.FeatureGroupSubGroup( grupo_ray, "Poco Riesgo")



map = folium.Map(location=[4.5, -74], tiles="Cartodb Positron", zoom_start=6)

def style_function(feature):
    return {
        "fillOpacity": 0.1,
        "weight": 1,
    }

folium.GeoJson(bog, style_function=style_function,
               popup = folium.GeoJsonPopup(fields=["uplnombre"])
              ).add_to(grupo_bog)

folium.GeoJson(cun, style_function=style_function,
               popup = folium.GeoJsonPopup(fields=["munnombre"])
              ).add_to(grupo_cun)
"""
folium.GeoJson(
    rc_indi_buena,
    marker = folium.Circle(radius = 10, fill_color="green", fill_opacity=0.4, color="green", weight=3),
    popup = folium.GeoJsonPopup(fields=["COOR_X", "COOR_Y"]),
    ).add_to(grupo)

folium.GeoJson(
    rc_indi_mala,
    marker = folium.Circle(radius = 10, fill_color="red", fill_opacity=0.4, color="red", weight=3),
    popup = folium.GeoJsonPopup(fields=["COOR_X", "COOR_Y"]),
    ).add_to(grupo2)

folium.GeoJson(
    sz_indi_buena,
    marker = folium.Circle(radius = 10, fill_color="green", fill_opacity=0.4, color="green", weight=3),
    popup = folium.GeoJsonPopup(fields=["COOR_X", "COOR_Y"]),
    ).add_to(grupo3)

folium.GeoJson(
    sz_indi_mala,
    marker = folium.Circle(radius = 10, fill_color="red", fill_opacity=0.4, color="red", weight=3),
    popup = folium.GeoJsonPopup(fields=["COOR_X", "COOR_Y"]),
    ).add_to(grupo4)
"""
#................................Parte de los Rayos....................................................
folium.GeoJson(
    ray_indi_buena,
    marker = folium.Circle(radius = 10, fill_color="green", fill_opacity=0.4, color="green", weight=3),
    popup = folium.GeoJsonPopup(fields=["COOR_X", "COOR_Y"]),
    ).add_to(grupo6)

folium.GeoJson(
    ray_indi_mala,
    marker = folium.Circle(radius = 10, fill_color="red", fill_opacity=0.4, color="red", weight=3),
    popup = folium.GeoJsonPopup(fields=["COOR_X", "COOR_Y"]),
    ).add_to(grupo5)


grupo_ray.add_to(map)
map.add_child(grupo5)
map.add_child(grupo6)
"""
grupo_sz.add_to(map)
map.add_child(grupo3)
map.add_child(grupo4)
grupo_rc.add_to(map)
map.add_child(grupo)
map.add_child(grupo2)
"""
grupo_bog.add_to(map)
grupo_cun.add_to(map)



folium.LayerControl(collapsed=False).add_to(map)
#----CAMBIAR----#
map.save('ray_18_ago.html')

In [ ]:
# Se carga el dataframe generado previamente
#----CAMBIAR----#
df3 = gpd.read_file('ray_18_ago.geojson')

# Se carga el geojson de los mapas de bogota por upz y cundinamarca por municipio
with open('bocu.geojson') as f:
    js = geojson.load(f)

# A cada rayo se le adiciona una columna llamada 'ubicacion' donde se identifica la upz o municipio al
# cual corresponde según sus coordenadas 

from shapely.geometry import shape, Point

df3['ubicacion'] = "otra"

for feature in js['features']:
    polygon = shape(feature['geometry'])
    for i in range(0, len(df3)):
        if polygon.contains(df3.loc[i, 'geometry']):
            df3.at[i,'ubicacion'] = feature["properties"]["nombre"]     

In [ ]:
# Se genera la base 'ubi' donde se consignan la cantidad de equipos con 
#nivel ceraunico alto, bajo y Nan(sin informacion)

bocu = gpd.read_file('bocu.geojson')
ubi = pd.DataFrame({'upz_o_municipio': ['x'],'cera_bueno': [0], 'cera_malo': [0]})

for upz_municipio, i in zip(bocu['nombre'], range(0, len(bocu)) ):
    cera = (df3[df3['ubicacion'].str.contains(upz_municipio)])['color'].value_counts()
    ubi.at[i, 'upz_o_municipio'] = upz_municipio
    
    try:
        ubi.at[i, 'cera_bueno'] = cera['green']
    except:
        ubi.at[i, 'cera_bueno'] = 0
    try:    
        ubi.at[i, 'cera_malo'] = cera['red']
    except:     
        ubi.at[i, 'cera_malo'] = 0
        

In [ ]:
# Se generan las columnas 'Hi_indi' y 'Hi_falla' donde se consigna el HI del municipio o upz según los datos
# de fallas e indisponibilidades de RC y SZ que albergan

for i in range(0, len(ubi)): 
        ubi.at[i, 'nivel_cer'] = (ubi.loc[i, 'cera_bueno']) / (ubi.loc[i, 'cera_malo'] + ubi.loc[i, 'cera_bueno'])
    
# Nota: al correr esta línea, se lanzan dos RuntimeWarning que no tiene mayor importancia

In [ ]:
# Ahora a la base con los RC y SZ 'df' se le adiciona el HI según la informacion histórica de indisponibilidades y fallas
# de acuerdo con la upz o municipio al que pertenece, se llama 'Hi_histo_ubi'
# Ojo: Recordar que ese HI está asociada a la upz o municipio, no al equipo

for i in range(0, len(df3)):
    for j in range(0, len(ubi)):
        if  df3.loc[i, 'ubicacion'] == ubi.loc[j, 'upz_o_municipio']:
            df3.at[i, 'nivel_cer_ubi'] = (ubi.loc[j, 'nivel_cer'])*100

ubi['numero_Rayos']=ubi['cera_bueno']+ubi['cera_malo']

In [ ]:

ubi

In [ ]:
"""
# Rangos: Buena 100 - 71, Aceptable 70 - 36, Mala 35 - 0
# Rayos: Poco Riesgo 0-50, Alto Riesgo 50-100
# Nota... El rango de indisponibilidades está con signo negativo por limitantes de np.interp, sin embargo,
#          también se cambia el signo del numero a interpolar para compensar

rangos = [100, 71, 35, 0]
rangos_rayos = [0,25,50,75,100]

# Se calcula el HI según las fallas e indisponibilidades
#for i in range(0 ,len(df3)):
#    df3.at[i ,'nivel_cer_ray'] = np.interp(df.loc[i ,'Numero_Fallas'] , rangos_fallas, rangos)  
"""

In [ ]:
#---CAMBIAR---#
df3.to_excel('nivel_ceraunico_18_ago.xlsx')
#ubi.to_excel('ubi_18_ago.xlsx')

In [ ]:
#Para generar el mapa de HI segun los historicos de fallas e indisponibilidades
#Mapa 1: De acuerdo a la intensidad de corriente de los rayos
import folium

bocu['nivel_cer_ubi'] = (ubi['nivel_cer'])*100

m = folium.Map(location=[4.7, -74], tiles="Cartodb Positron", zoom_start=8)

folium.Choropleth(
    bocu, data=bocu,
    columns=["nombre", "nivel_cer_ubi"],
    key_on="feature.properties.nombre",
    fill_color="RdYlGn",
    legend_name="Niver ceráunico (NC) por zona",
    bin=20
     ).add_to(m)

folium.GeoJson(
    bocu, 
    popup = folium.GeoJsonPopup(fields=["nombre","nivel_cer_ubi"], aliases=["Nombre", "NC por ubicación"] ),
    style_function=lambda feature: {"weight": 0,"fillOpacity": 0},
    tooltip = folium.GeoJsonTooltip(fields=["nombre"])
                  ).add_to(m)
#---CAMBIAR---#
m.save('NC_18_ago.html')

In [ ]:
#MAPA 2: De acuerdo al numero de rayos en cada zona
bocu['num_rayos'] = ubi['numero_Rayos']

m2 = folium.Map(location=[4.7, -74], tiles="Cartodb Positron", zoom_start=8)

folium.Choropleth(
    bocu, data=bocu,
    columns=["nombre", "num_rayos"],
    key_on="feature.properties.nombre",
    fill_color="YlOrRd",
    legend_name="Número de rayos por zona",
     ).add_to(m2)

folium.GeoJson(
    bocu, 
    popup = folium.GeoJsonPopup(fields=["nombre","num_rayos"], aliases=["Nombre", "No. rayos por ubicación"] ),
    style_function=lambda feature: {"weight": 0,"fillOpacity": 0},
    tooltip = folium.GeoJsonTooltip(fields=["nombre"])
                  ).add_to(m2)
#---CAMBIAR---#
m2.save('rayNum_18_ago.html')